In [173]:
from pprint import pprint
import random

In [199]:
def create_user_interest_dict(*interests):
    interest_count = len(interests)
    interest_weight = round((100-10) / interest_count,2) # Subtract 10 for 'Random'
    user_interests = {interest: interest_weight for interest in interests}
    user_interests['Random'] = 10 # Add 'Random' with constant value of 10
    return user_interests

In [180]:
# HELPER FUNCTIONS

def observeActionsTaken(video_tag, liked, shared, watched, loop_count):
    """
    This function calculates the total points based on the user's actions.

    Parameters:
    video_tag (str): The tag of the video.
    liked (bool): Whether the user liked the video.
    shared (bool): Whether the user shared the video.
    watched (bool): Whether the user watched the video.
    loop_count (int): The number of times the user looped the video.

    Returns:
    float: The total points calculated based on the user's actions.
    """
    # Actions and their corresponding points to add
    Actions = {
        "Like": 0.1,
        "Share": 0.2,
        "Watch": 0.1,  # more than 50% of total duration
        # todo: experiment with increasing watch value
        "Loop": 0.05,
    }

    if liked:
        total_points += Actions["Like"]
    if shared:
        total_points += Actions["Share"]
    if watched:
        total_points += Actions["Watch"]
    total_points += loop_count * Actions["Loop"]

    print(f"\nUpdating weights for video tag: {video_tag}")
    print(f"Total points to add: {total_points}")

    return total_points

def handleUpdateScore(user_dict_interest, video_tag, total_points):
    """
    This function updates the user's interest score based on the total points.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.
    total_points (float): The total points calculated based on the user's actions.

    Returns:
    dict: The updated user's interest dictionary.
    """
    # Increase the weight of the interest based on the action points
    user_dict_interest[video_tag] += total_points
    print(f"Updated {video_tag} to {user_dict_interest[video_tag]}\n")

    # Deduct the total points proportionally from existing interests (excluding 'Random' and the video_tag itself)
    total_deductable_weight = sum(
        weight for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag

    )
    print(f"Total weight to deduct from other interests: {total_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}")

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (total_points / total_deductable_weight) * user_dict_interest[interest]
            user_dict_interest[interest] -= deduction

            print(f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}")

    return user_dict_interest

def handleAddNewTag(user_dict_interest, video_tag):
    """
    This function adds a new interest to the user's interest dictionary.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.

    Returns:
    dict: The updated user's interest dictionary with the new interest added.
    """
    # Add new interest if it doesn't exist and give initial points
    initial_points = 5
    user_dict_interest[video_tag] = initial_points
    print(f"Added new interest {video_tag} with initial weight: {initial_points}")

    # Deduct initial points proportionally from existing interests (excluding 'Random')
    total_deductable_weight = sum(
        weight
        for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag
    )

    print(f"Total weight to deduct from other interests: {initial_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}")

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (
                initial_points / total_deductable_weight
            ) * user_dict_interest[interest]
            user_dict_interest[interest] -= deduction

            pprint(f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}")

    return user_dict_interest

In [194]:
def update_weight(
    user_dict_interest,
    video_tag,
    liked=False,
    shared=False,
    watched=False,
    loop_count=0,
):

    total_points = observeActionsTaken(video_tag, liked, shared, watched, loop_count)

    if video_tag in user_dict_interest:
        handleUpdateScore(user_dict_interest, video_tag, total_points)
    else:
        handleAddNewTag(user_dict_interest, video_tag)

    # Check total weight to ensure it sums to 100 (for debugging purposes)
    total_weight = sum(user_dict_interest.values())
    print(f"Total weight after update: {total_weight} (should be 100)\n")
    print(f"Updated interests: {user_dict_interest}\n")

    return user_dict_interest

In [230]:
def get_random_interest(user_dict_interest):
    random_value = round(random.uniform(0, 100), 0)
    print("_"*50)
    print(f"\nRandom value: {random_value}\n")
    cumulative_probability = 0.0
    visualization = []

    for interest, probability in user_dict_interest.items():
        previous_cumulative_probability = cumulative_probability
        cumulative_probability += probability
        visualization.append(
            (interest, previous_cumulative_probability, cumulative_probability)
        )
        pprint(
            f"Checking interest: {interest}, cumulative range: {previous_cumulative_probability} - {cumulative_probability}"
        )
        if random_value <= cumulative_probability:
            print(f"Selected interest: {interest}\n")
            break

    return interest

In [203]:
# New user selects interests (max 5)
# 10 total - 5 start to discover other 5 naturally

carl_interests = create_user_interest_dict(
    "DevOps", "Tests (SAT)", "Machine Learning", "Cats", "Cooking"
)
pprint(carl_interests)

{'Cats': 18.0,
 'Cooking': 18.0,
 'DevOps': 18.0,
 'Machine Learning': 18.0,
 'Random': 10,
 'Tests (SAT)': 18.0}


In [214]:
# Simulate user actions
# pprint(f"Before update: {carl_interests}")

update_weight(carl_interests, "NewTag", liked=True, watched=True, shared=True, loop_count=2)

# pprint(f"After update: {carl_interests}")


Updating weights for video tag: NewTag
Total points to add: 0.5
Updated NewTag to 8.5

Total weight to deduct from other interests: 0.5
Total deductable weight (excluding 'Random' and NewTag): 82.00000000000001
Deducted 0.1 from DevOps, new weight: 16.3
Deducted 0.1 from Tests (SAT), new weight: 16.3
Deducted 0.1 from Machine Learning, new weight: 16.3
Deducted 0.1 from Cats, new weight: 16.3
Deducted 0.1 from Cooking, new weight: 16.3
Total weight after update: 100.0 (should be 100)

Updated interests: {'DevOps': 16.3, 'Tests (SAT)': 16.3, 'Machine Learning': 16.3, 'Cats': 16.3, 'Cooking': 16.3, 'Random': 10, 'NewTag': 8.5}



{'DevOps': 16.3,
 'Tests (SAT)': 16.3,
 'Machine Learning': 16.3,
 'Cats': 16.3,
 'Cooking': 16.3,
 'Random': 10,
 'NewTag': 8.5}

In [234]:
# Simulate algorithm picking the next video to recommend via tag
selected_interest = get_random_interest(carl_interests)

__________________________________________________

Random value: 64.0

'Checking interest: DevOps, cumulative range: 0.0 - 16.3'
'Checking interest: Tests (SAT), cumulative range: 16.3 - 32.6'
('Checking interest: Machine Learning, cumulative range: 32.6 - '
 '48.900000000000006')
'Checking interest: Cats, cumulative range: 48.900000000000006 - 65.2'
Selected interest: Cats

